In [ ]:
def load_data(DATA_PATH):
    X = []
    Y = []
    for root, _, file_names in os.walk(spectrograms_path):
        for file_name in file_names:
            file_path = os.path.join(root, file_name)
            spectrogram_img = np.load(file_path[-1]) # (n_bins, n_frames, 1)
            x_train.append(spectrogram_img)
            labels = np.load(file_path[2])
    X = np.array(X)
    Y = np.array(Y)
    X = X[..., np.newaxis] 
    return X, Y

In [ ]:
def prepare_datasets(test_size, validation_size):
    """Loads data and splits it into train, validation and test sets.
    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split
    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    # load data
    X, Y = load_data(DATA_PATH)

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # add an axis to input sets
    X_train = X_train[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test

In [ ]:
def Model:
    
    images = keras.layers.Input(x_train.shape[1:])
    
    x = keras.layers.Conv2D(filters=16, kernel_size=[3, 3], padding='same')(images)
    block = keras.layers.BatchNormalization()(x)
    block = keras.layers.Activation("relu")(block)

    # ResNet block 2a
    block2 = keras.layers.Conv2D(filters=16, kernel_size=[3, 3], padding='same')(block)
    block2 = keras.layers.BatchNormalization()(block2)
    block2 = keras.layers.Activation("relu")(block2)

    # ResNet block 3a
    block3 = keras.layers.Conv2D(filters=16, kernel_size=[3, 3], padding='same')(block2)
    block3 = keras.layers.BatchNormalization()(block3)
    block3 = keras.layers.Activation("relu")(block3)

    #inio Squeeze and Excitation 1a
    sq = keras.layers.GlobalAveragePooling2D()(block3)
    sq = keras.layers.Reshape((1,1,16))(sq)
    sq = keras.layers.Dense(units=16,activation="relu")(sq)
    sq = keras.layers.Dense(units=16,activation="sigmoid")(sq)
    block = keras.layers.multiply([block3,sq])

    #Res block
    x = keras.layers.Conv2D(filters=16, kernel_size=[3, 3], padding='same')(images)
    x = keras.layers.BatchNormalization()(x)

    #final Output
    net = keras.layers.add([x,block])
    net = keras.layers.Activation("relu")(net)

    net = Flatten()(net)
    net = Dropout(0.5)(net) 
    x = Dense(6)(net)
    net = keras.layers.Activation("relu")(x)
    
    return model

In [ ]:
print(model.summary())

In [ ]:
def train(x_train, y_train, learning_rate, batch_size, epochs):
    
    checkpointer = keras.callbacks.ModelCheckpoint(filepath = 'cnn_from_scratch_fruits.hdf5', verbose = 1, save_best_only = True)
    earlystopper = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
    train_model = model.fit(x_train, y_train, batch_size, learning_rate, epochs, callbacks = [checkpointer,earlystopper], shuffle=True)
    return train_model

In [ ]:
def plot_history(history):
    """Plots accuracy/loss for training/validation set as a function of the epochs
        :param history: Training history of model
        :return:
    """

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # create error sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.show()

In [ ]:
if __name__ == "__main__":
    
    # get train, validation, test splits
    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

    model = keras.models.Model(inputs=images,outputs=net)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    model.summary()

    # train model
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

    # plot accuracy/error for training and validation
    plot_history(history)

    # evaluate model on test set
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print('\nTest accuracy:', test_acc)
